In [1]:
import neptune.new as neptune
from neptune.new.types import File

run = neptune.init(
        project='arsde/J21SYSU',
        run='JSYSU-1',
        tags=['J21', '1M', 'all features', 'real', 'tun'],
        name='run'
)

https://app.neptune.ai/arsde/J21SYSU/e/JSYSU-1
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [2]:
import os

def init(run_name):
    os.system('mkdir {}'.format(run_name))
    os.system('mkdir {}/models_saved'.format(run_name))
    os.system('mkdir {}/results'.format(run_name))
#     os.system('mkdir {}/preds'.format(run_name))
    
run_name = input()
init(run_name)

JSYSU-1


In [4]:
import numpy as np
import pandas as pd
from scipy.stats import norm                                                                                                                                                                                                                                                                                                                                                                                                                   
from scipy.optimize import curve_fit
from scipy import stats

import plotly.graph_objs as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

import plotly.io as pio
pio.templates.default = 'plotly_white'

import warnings
warnings.filterwarnings("ignore")

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18198439486551863554
]


In [6]:
bdt_J19 = pd.read_csv('~/J21_SYSU/df_bdt_eval.csv')
bdt_J19_5M_23 = bdt_J19[(bdt_J19['opt']==23) & (bdt_J19['model']=='5M')]
bdt_J19_5M_0 = bdt_J19[(bdt_J19['opt']==0) & (bdt_J19['model']=='5M')]

In [8]:
path='/mnt/cephfs/ml_data/mc_2021/'

data_real = pd.read_csv('{}processed_data/ProcessedTrainReal/ProcessedTrain_1M.csv.gz'.format(path))
data_real = data_real[data_real['edepR'] < 17.2]

In [9]:
all_data = [data_real]

In [10]:
data_real

,AccumCharge,nPMTs,R_cc,pho_cc,x_cc,y_cc,z_cc,gamma_z_cc,gamma_y_cc,gamma_x_cc,...,pe_95p,pe_96p,pe_97p,pe_98p,pe_99p,edep,edepX,edepY,edepZ,edepR
0,16795.398556,6698.0,12.025256,8.872896,7.972470,-3.894611,8.116558,0.914759,-0.342320,0.885576,...,8.531833,10.440701,12.698865,15.875377,21.189132,8.309155,10.414397,-5.163487,10.787749,15.858651
1,6658.966649,3645.0,10.909448,10.318122,-8.104657,6.385779,-3.542939,-0.343370,0.721947,-1.109805,...,5.868211,6.806834,7.978919,10.161503,12.634588,3.298830,-12.139917,9.371063,-5.497660,16.291675
2,7296.981869,4395.0,10.514288,9.325840,-1.437017,9.214460,-4.855816,-0.520684,1.819606,-0.137967,...,4.598118,5.096357,5.661463,6.343645,7.667880,3.555040,-1.927146,12.013859,-6.652611,13.867370
3,14586.374299,8350.0,6.747823,4.152723,0.959516,-4.040351,5.318648,1.280762,-0.747589,0.143656,...,4.278572,4.571887,5.073629,5.766348,6.876392,7.941365,1.193331,-5.172269,6.760569,8.595446
5,11599.028010,5631.0,11.202894,3.015708,-1.945551,-2.304198,10.789362,3.577721,-0.210172,-0.176345,...,6.491171,7.242658,8.537913,10.235875,12.480641,6.084913,-2.486934,-3.070554,14.368409,14.901823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1249981,7337.626369,5533.0,2.561501,2.397117,-0.488978,2.346715,-0.902839,-0.376636,2.285573,-0.194471,...,2.808414,3.012805,3.346317,3.855301,5.018902,3.990185,-0.701865,2.894827,-1.457681,3.316244
1249982,19017.877450,7274.0,11.857567,9.747107,1.315855,-9.657879,6.752465,0.692766,-1.403867,0.111661,...,8.544738,10.150525,13.064990,17.339913,26.712126,9.868357,1.838813,-13.133485,9.416347,16.264601
1249983,3698.755551,2710.0,9.393078,6.892417,6.878206,-0.442373,6.381575,0.925883,-0.047148,1.075242,...,3.087671,3.352464,3.769108,4.331282,5.607043,1.807647,9.715888,-0.359872,9.423583,13.540011
1249984,4615.040152,2578.0,9.874959,6.038693,5.964950,-0.940841,-7.813387,-1.293887,-0.095711,0.757952,...,5.772201,6.907113,8.386624,10.717686,14.164438,2.405519,10.338062,-1.435942,-13.546845,17.101300


In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X = data_real.iloc[:, :-5]
scaler.fit(X)

X = scaler.transform(X)
y = data_real['edep']

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

def build_model(hp):
    model = Sequential()
    
    units_input = hp.Int('units_input', min_value=16, max_value=128, default=32)
    units_in_hidden_layer = hp.Int('units_in_hidden_layer', min_value=16, max_value=128, default=32)
    num_hidden_layers = hp.Int('num_hidden_layers', min_value=1, max_value=16, default=6)
    
    activation = hp.Choice('activation', values=['relu', 'elu', 'selu'])  
    
    optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop', 'SGD'])
    
    lr = hp.Float('lr', min_value=1e-4, max_value=1e-2, default=1e-3, sampling='LOG')
    deray_lr = hp.Choice('deray_lr', [0.0, -1.0])
    
    kernel_initializer_input = hp.Choice('kernel_initializer_input', ['normal', 'lecun_normal', 'uniform'])
    kernel_initializer_hidden = hp.Choice('kernel_initializer_hidden', ['normal', 'lecun_normal', 'uniform'])
    
    if deray_lr == -1.0:
        decay_steps = hp.Int('decay_steps', min_value=500, max_value=20000, default=2000)
        decay_rate = hp.Float('decay_rate', min_value=0.1, max_value=0.9, default=0.8)

        lr = tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=lr,
            decay_steps=decay_steps,
            decay_rate=decay_rate
        )


    model.add(Dense(
                units=units_input,
                input_dim=X.shape[1],
                kernel_initializer=kernel_initializer_input,
                activation=activation
                )
            )
    
    for i in range(num_hidden_layers):
        model.add(Dense(
            units=units_in_hidden_layer,
            kernel_initializer=kernel_initializer_hidden,
            activation=activation
            )
        )   
                
    if optimizer == 'adam':
        optimizer = Adam(lr)
    elif optimizer == 'rmsprop':
        optimizer = RMSprop(lr)
    elif optimizer == 'SGD':
        optimizer = SGD(lr)
        
    model.add(Dense(1, kernel_initializer='normal', activation='linear'))
    
    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=[
                tf.keras.metrics.MeanAbsolutePercentageError(name='mape'),
                'mse',
                'mae'
            ])
    
    return model

In [16]:
import neptunecontrib.monitoring.kerastuner as npt_utils
from kerastuner.tuners import BayesianOptimization

tuner = BayesianOptimization(
    build_model,
    objective='val_mse',
    max_trials=150,
    directory='saved_networks',
    project_name='FCDNN_tuning_1M_AF',
    logger=npt_utils.NeptuneLogger()
)

In [17]:
from tensorflow.keras.callbacks import EarlyStopping

monitor = EarlyStopping(monitor='val_mse', patience=5, mode='min',
        restore_best_weights=True)

In [27]:
tuner.search(X,
             y,
             batch_size=512,
             epochs=100, 
             validation_split=0.2,
             callbacks=[monitor],
             verbose=2
)


Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
units_input       |110               |?                 
units_in_hidden...|83                |?                 
num_hidden_layers |4                 |?                 
activation        |selu              |?                 
optimizer         |adam              |?                 
lr                |0.0028585         |?                 
deray_lr          |-1                |?                 
kernel_initiali...|uniform           |?                 
kernel_initiali...|normal            |?                 
decay_steps       |2000              |?                 
decay_rate        |0.8               |?                 

Epoch 1/100
1793/1793 - 17s - loss: 0.2792 - mape: 4.6240 - mse: 0.2792 - mae: 0.2115 - val_loss: 0.0268 - val_mape: 2.7597 - val_mse: 0.0268 - val_mae: 0.1310
Epoch 2/100


KeyboardInterrupt: 

In [ ]:
tuner.results_summary()

In [ ]:
npt_utils.log_tuner_info(tuner)

In [ ]:
models = tuner.get_best_models(1)
pickle.dump(models, open("{}/models_saved/fcdnn_real.dat".format(run_name), "wb"))

In [ ]:
y_true_array = []
y_pred_array = []
for j in range(len(all_data)):
    if j == 0:
        name = 'ProcessedTestIdeal'
    elif j == 1:
        name = 'ProcessedTestReal'
    y_true = []
    y_pred = []
    for i in range(11):
        test = pd.read_csv('{}processed_data/{}/{}MeV.csv.gz'.format(path, name, i))
        test = test[test['edepR'] < 17.2]
        edep = np.array(test['edep'])
        X_test = test.iloc[:, :-5]
        edep_preds = models.predict(np.array(X_test))

        y_true.append(edep)
        y_pred.append(edep_preds)
    y_true_array.append(y_true)
    y_pred_array.append(y_pred)

In [ ]:
diffs = np.array([
    [y_pred_array[j][i] - y_true_array[j][i] for i in range(len(y_pred_array[0]))]
    for j in range(len(y_pred_array))
])

In [ ]:
names = ['real']

In [ ]:
energies = np.array([1.022+i for i in range(11)]).round(5)
energies

In [ ]:
a_array = []
errors_array = []
for k in range(diffs.shape[0]):
    a = []
    e = []
    for i in range(diffs.shape[1]):
        fig, ax = plt.subplots()
        nbins = 150
        n, bins, patches = ax.hist(diffs[k][i], nbins, density=True, facecolor = 'grey', alpha = 0.5, label='before');
        plt.close(fig)
        centers = (0.5*(bins[1:]+bins[:-1]))
        pars, cov = curve_fit(lambda x, mu, sig : norm.pdf(x, loc=mu, scale=sig), centers, n, p0=[0,1])  
        a.append(pars)
        e.append(cov)
    a_array.append(a)
    errors_array.append(e)

In [ ]:
for k in range(len(all_data)):
    fig = go.Figure()

    for i in range(len(diffs[k])): 
        x = np.linspace(diffs[k][i][:25000].min(), diffs[k][i][:25000].max(), 100)
        p = stats.norm.pdf(x, a_array[k][i][0], a_array[k][i][1])

        fig.add_trace(go.Scattergl(x=x,
                     y=p, mode='lines', name='mu={:.3f} +- {:.3f}, sigma={:.3f} +- {:.3f}'.format(
                         a_array[k][i][0], np.sqrt(errors_array[k][i][0][0]),
                         a_array[k][i][1], np.sqrt(errors_array[k][i][1][1])),
                        visible = (i==0)
                    ))

    for i in range(len(diffs[k])): 
        fig.add_trace(go.Histogram(x=diffs[k][i][:25000], xbins=dict(size=0.005),
                      showlegend=False, histnorm='probability density',
                     visible = (i==0)
                    ))

    buttons = []
    for N in range(0, len(diffs[k])): 
        buttons.append(
            dict(
                 args=['visible', [False]*N + [True] + [False]*(len(diffs[k])-1-N)],
                     label='Energy =  {} MeV'.format(energies[N]),
                 method='restyle'
            )
        )
        
    fig.update_layout(

        xaxis = dict(
            showline=True,
            ticks='outside',
            mirror=True,
            linecolor='black',
            showgrid=True,
            gridcolor='grey',
            gridwidth=0.25,
        ),

        yaxis = dict(
            showline=True,
            ticks='outside',
            mirror=True,
            linecolor='black',
            tick0=0,
#             dtick=1,
            showgrid=True,
            gridcolor='grey',
            gridwidth=0.25,
            zeroline=True,
            zerolinecolor='black',
            zerolinewidth=0.25
            ),
    )

    fig.update_layout(
        title = '{}'.format(names[k]),
        xaxis_title=r"$$E_{rec} - E_{true}$$",
        showlegend=True,
        updatemenus=list([
            dict(
                x=0.5,
                y=1.2,
                yanchor='top',
                buttons=buttons
            ),
        ]),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.05,
            xanchor="right",
            x=1
        )
    )
    
    fig.show()
    run['output/Result_distributions.pdf'].upload(File.as_html(fig))

In [ ]:
def plot_results(appr=False):
        fig = make_subplots(rows=2, cols=1,
                        shared_xaxes=True,
                        vertical_spacing=0.01,
                        row_width=[0.25, 0.75]
    )

    for k in range(diffs.shape[0]):
        fig.add_trace(
            go.Scatter(
                x=energies,
                y=res[k],
                mode='markers',
                marker=dict(
                    color=colors[k],
                    symbol=symbols[k]
                ),
                showlegend=True,
                error_y=dict(
                    type='data',
                    width=10,
                    array=error_sigma[k],
                    visible=True
                ),
                name=names[k]
            ), row=1, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=energies,
                y=bias[k],
                mode='markers',
                showlegend=False,
                marker=dict(
                    color=colors[k],
                    symbol=symbols[k]
                ),
                error_y=dict(
                        type='data',
                        width=10,
                        array=error_mu[k],
                        visible=True
                ),
                name=names[k]
            ), row=2, col=1
        )

    if appr:
        for k in range(len(names)):
            fig.add_trace(
                go.Scatter(
                    x=x_lin,
                    y=func(x_lin, a[k], b[k], c[k]),
                    mode='lines',
                    line=dict(
                    ),
                    opacity=0.5,
                    showlegend=False,
                    name=names[k],
                    marker=dict(
                        color=colors[k]
                    )
                ), row=1, col=1
            )

    trace = lambda x, y, error_y, col, name, sym, leg=True: go.Scatter(
                        x=x,
                        y=100*y,
                        mode='markers',
                        name=name,
                        showlegend=leg,
                        marker=dict(
                            color=col,
                            symbol=sym
                        ),
                        error_y=dict(
                            type='data',
                            width=10,
                            array=100*error_y
                        )
    )

    fig.add_trace(
        trace(
            bdt_J19_5M_0.energy+1.022,
            bdt_J19_5M_0.res,
            bdt_J19_5M_0.res_err,
            'blue',
            'J19 5M Ideal',
            'square',
        ),
        row=1, col=1
    )

    fig.add_trace(
        trace(
            bdt_J19_5M_23.energy+1.022,
            bdt_J19_5M_23.res,
            bdt_J19_5M_23.res_err,
            'green',
            'J19 5M Real',
            'cross-open-dot',
        ),
        row=1, col=1
    )

    fig.add_trace(
        trace(
            bdt_J19_5M_0.energy+1.022,
            bdt_J19_5M_0.bias,
            bdt_J19_5M_0.bias_err,
            'blue',
            'J19 5M Ideal',
            'square',
            False
        ),
        row=2, col=1
    )

    fig.add_trace(
        trace(
            bdt_J19_5M_23.energy+1.022,
            bdt_J19_5M_23.bias,
            bdt_J19_5M_23.bias_err,
            'green',
            'J19 5M Real',
            'cross-open-dot',
            False
        ),
        row=2, col=1
    )

    xaxis = dict(
        showline=True,
        ticks='outside',
        mirror=True,
        tick0=1,
        dtick=1,
        linecolor='black',
        showgrid=True,
        gridcolor='grey',
        gridwidth=0.25,
    )

    yaxis = lambda range: dict(
        showline=True,
        ticks='outside',
        mirror=True,
        linecolor='black',
        range=range,
        showgrid=True,
        gridcolor='grey',
        gridwidth=0.25,
        zeroline=True,
        zerolinecolor='black',
        zerolinewidth=0.25
    )

    fig.update_layout(
        xaxis2_title="Visible energy, MeV",
        yaxis1_title="Resolution, %",
        yaxis2_title="Bias, %",

        xaxis1 = xaxis,
        xaxis2 = xaxis,
        yaxis1 = yaxis([0, 3.5]),
        yaxis2 = yaxis([-0.25, 0.25]),

        showlegend=True,
        font=dict(
                family="Times New Roman",
                size=18,
        ),
        legend=dict(
            x=0.75,
            y=0.99,
            title_font_family="Times New Roman",
            font=dict(
                family="Times New Roman",
                size=18,
                color="black"
            ),
            bordercolor="Black",
            borderwidth=2
        )
    )

    fig.show()
    if appr:
        pio.write_image(fig, '{}/results/appr_results.pdf'.format(run_name), width=900, height=600)
        run['output/appr_results.pdf'].upload(File.as_html(fig))
    else:
        pio.write_image(fig, '{}/results/results.pdf'.format(run_name), width=900, height=600)
        run['output/results.pdf'].upload(File.as_html(fig))

In [ ]:
colors = ['darkviolet', 'blue', 'green']
symbols = ['star-square']

In [ ]:
error_sigma = []
for k in range(diffs.shape[0]):
    error = [100 * np.sqrt(errors_array[k][i][1][1]) / energies[i] for i in range(len(energies))]
    error_sigma.append(error)
    
error_mu = []
for k in range(diffs.shape[0]):
    error = [100 * np.sqrt(errors_array[k][i][0][0]) / energies[i] for i in range(len(energies))]
    error_mu.append(error)

res = []
bias = []
for k in range(diffs.shape[0]):
    sigma = [100 * a_array[k][i][1] / energies[i] for i in range(len(energies))]
    mu = [100 * a_array[k][i][0] / energies[i] for i in range(len(energies))]
    res.append(sigma)
    bias.append(mu)
    
plot_results()

In [ ]:
def a(x, a):
    return np.sqrt((a/x**0.5)**2)


def b(x, b):
    b_list = []
    b_list.append(np.sqrt(b**2))
    return b_list*len(x)


def c(x, c):
    return np.sqrt((c/x)**2)


def func(x, a, b, c):
    return np.sqrt((a/x**0.5)**2 + b**2 + (c/x)**2) 


def approximated(x, y, yerr):
    popt, pcov = curve_fit(func, x, y, sigma=yerr, maxfev=10**9, bounds=([0, 0, 0], [5, 5, 5]))
    a, b, c = popt
    perr = np.sqrt(abs(pcov.diagonal()))

    return func(x, a, b, c), popt, perr

In [ ]:
y_approximated_array = []
coefs_array = []
errors_array = []
for i in range(diffs.shape[0]):
    y_approximated, coefs, errors = approximated(
        energies[1:10], res[i][1:10], error_sigma[i][1:10])
    y_approximated_array.append(y_approximated)
    coefs_array.append(coefs)
    errors_array.append(errors)
    
y_approximated, coefs, errors = approximated(
    bdt_J19_5M_0.energy+1.022, 100*bdt_J19_5M_0.res, 100*bdt_J19_5M_0.res_err)
y_approximated_array.append(y_approximated)
coefs_array.append(coefs)
errors_array.append(errors)

y_approximated, coefs, errors = approximated(
    bdt_J19_5M_23.energy+1.022, 100*bdt_J19_5M_23.res, 100*bdt_J19_5M_23.res_err)
y_approximated_array.append(y_approximated)
coefs_array.append(coefs)
errors_array.append(errors)

In [ ]:
reindex = [0, 3, 1, 4, 2, 5]
coefs_df = pd.DataFrame(
    np.hstack((coefs_array, errors_array))
)[reindex]
coefs_df.columns = ['a', r'$\Delta a$', 'b', r'$\Delta b$', 'c', r'$\Delta c$']

a = np.array(coefs_array).T[0]
b = np.array(coefs_array).T[1]
c = np.array(coefs_array).T[2]

In [ ]:
x_lin = np.linspace(0.8, 11.5, 1000)
plot_results(appr=True)

**The JUNO collaboration., Abusleme, A., Adam, T. et al. Calibration strategy of the JUNO experiment. J. High Energ. Phys. 2021, 4 (2021). https://doi.org/10.1007/JHEP03(2021)004**

It was found, numerically, that the JUNO baseline requirement to determine the MO to 3 – 4 $\sigma$ significance could be translated into a convenient requirement on an effective resolution $\tilde{a}$ as:

$$\tilde{a} \equiv \sqrt{(a)^2 + (1.6 \times b)^2 + \left(\frac{c}{1.6}\right)^2} \leq 3\%$$

Let's calculate the variance as follows:

$$\Delta \tilde{a} = \sqrt{\left(\frac{\partial \tilde{a}}{\partial a} \Delta a \right)^2 + \left(\frac{\partial \tilde{a}}{\partial b} \Delta b \right)^2 + \left(\frac{\partial \tilde{a}}{\partial c} \Delta c \right)^2}$$

Then:
$$\Delta \tilde{a} = \frac{\sqrt{\left(a\Delta a\right)^2 + \left(2.56\times b\Delta b\right)^2 + \left(\frac{c\Delta c}{2.56}\right)^2}}{\tilde{a}}$$

In [ ]:
coefs_df = coefs_df.round(3)
coefs_df.index = names
coefs_df[r'$\tilde{a}$'] = (coefs_df['a']**2 + (1.6 * coefs_df['b'])**2 + (coefs_df['c'] / 1.6)**2)**0.5 
coefs_df[r'$\Delta \tilde{a}$'] = np.sqrt( (coefs_df['a']*coefs_df[r'$\Delta a$'])**2 + \
                                           (2.56*coefs_df['b']*coefs_df[r'$\Delta b$'])**2 + \
                                           (coefs_df['c']*coefs_df[r'$\Delta c$'] / 2.56)**2) / coefs_df[r'$\tilde{a}$']

coefs_df

In [ ]:
run['coefs_df'].upload(File.as_html(coefs_df))
coefs_df